# 05 — Probabilistic forecasts (24-hour ahead)

This notebook repeats the probabilistic forecasting workflow from Notebook 04, but for a 24-hour-ahead
load forecasting horizon. Longer horizons generally have:

- larger point errors
- wider prediction intervals
- harder calibration (more regime shift / nonstationarity)

We compare:
- Quantile regression: direct conditional quantiles (80% band via q0.1–q0.9)
- Conformal prediction: residual-based 90% intervals around a point forecast

Metrics:
- Coverage and width (sharpness)
- MAE/RMSE (point accuracy)
- Quantile calibration and pinball loss
- Stress tests: extremes, peak week, hour-of-day diagnostics


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


from src.data import load_opsd_germany
from src.features import make_features
from src.models import train_point_model, train_quantile_models, predict_quantiles
from src.conformal import compute_absolute_residuals, conformal_interval
from src.evaluation import (
    regression_metrics,
    interval_coverage,
    interval_width,
    quantile_calibration,
    plot_quantile_calibration,
    pinball_loss,
)

plt.rcParams["figure.figsize"] = (12, 4)
plt.rcParams["axes.grid"] = True

In [2]:
raw = load_opsd_germany(str(PROJECT_ROOT / "data" / "time_series_60min_singleindex.csv"))
X, y = make_features(raw, horizon=1, target_col="load")

# 24-hour-ahead probabilistic forecasts: data and splits
X_24, y_24 = make_features(raw, horizon=24, target_col="load")

n24 = len(X_24)
cutoff_train_24 = X_24.index[int(0.6 * n24)]
cutoff_cal_24 = X_24.index[int(0.8 * n24)]

X_train_24 = X_24[X_24.index < cutoff_train_24]
y_train_24 = y_24[y_24.index < cutoff_train_24]

X_cal_24 = X_24[(X_24.index >= cutoff_train_24) & (X_24.index < cutoff_cal_24)]
y_cal_24 = y_24[(y_24.index >= cutoff_train_24) & (y_24.index < cutoff_cal_24)]

X_test_24 = X_24[X_24.index >= cutoff_cal_24]
y_test_24 = y_24[X_24.index >= cutoff_cal_24]

X_train_24.shape, X_cal_24.shape, X_test_24.shape


((29944, 28), (9981, 28), (9982, 28))

This trains 24h quantile models, computes 80% interval coverage/width, and pinball losses for q=0.1/0.5/0.9; compares side-by-side with the 1h results.


In [3]:
# 24h quantile regression: coverage, width, and pinball loss
quantiles = [0.1, 0.5, 0.9]
q_models_24 = train_quantile_models(X_train_24, y_train_24, quantiles=quantiles)
q_preds_test_24 = predict_quantiles(q_models_24, X_test_24)

lower_q_24 = q_preds_test_24["q_0.1"].values
upper_q_24 = q_preds_test_24["q_0.9"].values
median_q_24 = q_preds_test_24["q_0.5"].values

coverage_q_24 = interval_coverage(y_test_24, lower_q_24, upper_q_24)
width_q_24 = interval_width(lower_q_24, upper_q_24)

pin_q10_24 = pinball_loss(y_test_24.values, lower_q_24, 0.1)
pin_q50_24 = pinball_loss(y_test_24.values, median_q_24, 0.5)
pin_q90_24 = pinball_loss(y_test_24.values, upper_q_24, 0.9)

summary_24 = pd.DataFrame({
    "horizon": ["1h", "24h"],
    "coverage_80": [coverage_q, coverage_q_24],
    "avg_width_80": [width_q, width_q_24],
    "pinball_q10": [pin_q10, pin_q10_24],
    "pinball_q50": [pin_q50, pin_q50_24],
    "pinball_q90": [pin_q90, pin_q90_24],
})
summary_24


NameError: name 'coverage_q' is not defined

This constructs 90% conformal intervals for the 24h horizon and summarizes coverage, width, and point errors against the 80% quantile intervals.


In [ ]:
# 24h conformal intervals (target coverage 90%)
point_model_24 = train_point_model(X_train_24, y_train_24)
residuals_24 = compute_absolute_residuals(point_model_24, X_cal_24, y_cal_24)
alpha_24 = 0.1
lower_conf_24, upper_conf_24 = conformal_interval(model=point_model_24, X=X_test_24, residuals=residuals_24, alpha=alpha_24)

y_pred_point_test_24 = point_model_24.predict(X_test_24)

coverage_conf_24 = interval_coverage(y_test_24, lower_conf_24, upper_conf_24)
width_conf_24 = interval_width(lower_conf_24, upper_conf_24)

pd.DataFrame({
    "method": ["quantile_80_24h", "conformal_90_24h"],
    "coverage": [coverage_q_24, coverage_conf_24],
    "avg_width": [width_q_24, width_conf_24],
    "MAE": [regression_metrics(y_test_24, median_q_24)["MAE"], regression_metrics(y_test_24, y_pred_point_test_24)["MAE"]],
    "RMSE": [regression_metrics(y_test_24, median_q_24)["RMSE"], regression_metrics(y_test_24, y_pred_point_test_24)["RMSE"]],
})


At 24h, absolute errors and widths increase; conformal typically maintains target coverage better than direct quantiles.


This evaluates coverage and width on the top X% load hours for both horizons, revealing calibration under stress (peak demand).


In [ ]:
# Extreme-days slice: evaluate coverage/width on top X% of load
X_percent = 0.05  # top 5% by default

thr_1h = np.quantile(y_test.values, 1 - X_percent)
mask_1h = (y_test.values >= thr_1h)

ext_cov_q_1h = interval_coverage(y_test.values[mask_1h], lower_q[mask_1h], upper_q[mask_1h])
ext_width_q_1h = interval_width(lower_q[mask_1h], upper_q[mask_1h])

ext_cov_conf_1h = interval_coverage(y_test.values[mask_1h], lower_conf[mask_1h], upper_conf[mask_1h])
ext_width_conf_1h = interval_width(lower_conf[mask_1h], upper_conf[mask_1h])

thr_24h = np.quantile(y_test_24.values, 1 - X_percent)
mask_24h = (y_test_24.values >= thr_24h)

ext_cov_q_24h = interval_coverage(y_test_24.values[mask_24h], lower_q_24[mask_24h], upper_q_24[mask_24h])
ext_width_q_24h = interval_width(lower_q_24[mask_24h], upper_q_24[mask_24h])

ext_cov_conf_24h = interval_coverage(y_test_24.values[mask_24h], lower_conf_24[mask_24h], upper_conf_24[mask_24h])
ext_width_conf_24h = interval_width(lower_conf_24[mask_24h], upper_conf_24[mask_24h])

pd.DataFrame({
    "horizon": ["1h", "1h", "24h", "24h"],
    "method": ["quantile_80", "conformal_90", "quantile_80", "conformal_90"],
    "extreme_top_frac": [X_percent]*4,
    "coverage_on_extremes": [ext_cov_q_1h, ext_cov_conf_1h, ext_cov_q_24h, ext_cov_conf_24h],
    "avg_width_on_extremes": [ext_width_q_1h, ext_width_conf_1h, ext_width_q_24h, ext_width_conf_24h],
})


Notice lower coverage on extremes for quantile intervals and more stable coverage for conformal; widths generally increase on extreme hours.


This plots the highest-load week in the test set with both 80% quantile and 90% conformal bands for a direct visual comparison around the peak.


In [ ]:
# Plot a peak-load week with intervals (1h horizon)
peak_idx = int(np.argmax(y_test.values))
start_idx = max(0, peak_idx - 3*24)
end_idx = min(len(y_test), peak_idx + 4*24)
sl = slice(start_idx, end_idx)

plt.figure(figsize=(12, 4))
plt.plot(y_test.index[sl], y_test.values[sl], label="Actual", lw=1.5)
plt.plot(y_test.index[sl], median_q[sl], label="Median (q=0.5)", lw=1.0)
plt.fill_between(y_test.index[sl], lower_q[sl], upper_q[sl], alpha=0.25, label="80% quantile band")
plt.fill_between(y_test.index[sl], lower_conf[sl], upper_conf[sl], alpha=0.25, label="90% conformal band")
plt.title("Peak-load week: quantile (80%) vs conformal (90%) intervals – 1h ahead")
plt.legend()
plt.tight_layout()
plt.show()


This computes coverage and widths by hour of day for both 80% quantile and 90% conformal intervals, plus widths normalized by median load.


In [ ]:
# Hour-of-day coverage and normalized interval widths (1h horizon)
hour_idx = y_test.index.hour.values

# Quantile 80% by hour
df_hour_q = pd.DataFrame({
    "hour": hour_idx,
    "y": y_test.values,
    "lower": lower_q,
    "upper": upper_q,
})

cov_by_hour_q = df_hour_q.groupby("hour").apply(lambda g: interval_coverage(g["y"].values, g["lower"].values, g["upper"].values))
width_by_hour_q = df_hour_q.groupby("hour").apply(lambda g: interval_width(g["lower"].values, g["upper"].values))

# Conformal 90% by hour
df_hour_c = pd.DataFrame({
    "hour": hour_idx,
    "y": y_test.values,
    "lower": lower_conf,
    "upper": upper_conf,
})

cov_by_hour_c = df_hour_c.groupby("hour").apply(lambda g: interval_coverage(g["y"].values, g["lower"].values, g["upper"].values))
width_by_hour_c = df_hour_c.groupby("hour").apply(lambda g: interval_width(g["lower"].values, g["upper"].values))

median_load = float(np.median(y_test.values))
summary_hour = pd.DataFrame({
    "cov_q80": cov_by_hour_q,
    "cov_conf90": cov_by_hour_c,
    "width_q80": width_by_hour_q,
    "width_conf90": width_by_hour_c,
})
summary_hour["width_q80_norm"] = summary_hour["width_q80"] / median_load
summary_hour["width_conf90_norm"] = summary_hour["width_conf90"] / median_load
summary_hour.sort_index()


In [ ]:
# CRPS-style approximation via average pinball over a quantile grid (1h horizon)
q_grid = np.linspace(0.05, 0.95, 19)
q_grid_models = train_quantile_models(X_train, y_train, quantiles=list(q_grid))
q_grid_preds = predict_quantiles(q_grid_models, X_test)

# Compute average pinball across the grid
pinball_values = []
for q in q_grid:
    col = f"q_{q}"
    pinball_values.append(pinball_loss(y_test.values, q_grid_preds[col].values, float(q)))

crps_like_avg_pinball_1h = float(np.mean(pinball_values))

pd.DataFrame({
    "metric": ["avg_pinball_over_grid"],
    "value": [crps_like_avg_pinball_1h],
})
